In [2]:
import open3d as o3d
import numpy as np
import copy
import os
import sys


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# ICP registration
This tutorial demonstrates the ICP (Iterative Closest Point) registration algorithm. It has been a mainstay of geometric registration in both research and industry for many years. The input are two point clouds and an initial transformation that roughly aligns the source point cloud to the target point cloud. The output is a refined transformation that tightly aligns the two point clouds. A helper function `draw_registration_result` visualizes the alignment during the registration process. In this tutorial, we show two ICP variants, the point-to-point ICP and the point-to-plane ICP [\[Rusinkiewicz2001\]](../reference.html#rusinkiewicz2001).

## Helper visualization function
The function below visualizes a target point cloud and a source point cloud transformed with an alignment transformation. The target point cloud and the source point cloud are painted with cyan and yellow colors respectively. The more and tighter the two point clouds overlap with each other, the better the alignment result.

In [3]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #source_temp.paint_uniform_color([1, 0.706, 0])
    #target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])

<div class="alert alert-info">
    
**Note:** 

Since the functions `transform` and `paint_uniform_color` change the point cloud, we call `copy.deepcopy` to make copies and protect the original point clouds.

</div>

## Input
The code below reads a source point cloud and a target point cloud from two files. A rough transformation is given.

<div class="alert alert-info">
    
**Note:** 

The initial alignment is usually obtained by a global registration algorithm. See [Global registration](../pipelines/global_registration.rst) for examples.

</div>

In [10]:
source = o3d.io.read_point_cloud("PointClouds/Lab_Cam1.ply")
target = o3d.io.read_point_cloud("PointClouds/Lab_Cam2.ply")
threshold = 0.02
trans_init = np.identity(4)
#draw_registration_result(source, target, trans_init)

The function `evaluate_registration` calculates two main metrics:

- `fitness`, which measures the overlapping area (# of inlier correspondences / # of points in target). The higher the better.
- `inlier_rmse`, which measures the RMSE of all inlier correspondences. The lower the better.

In [11]:
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(
    source, target, threshold, trans_init)
print(evaluation)

Initial alignment
RegistrationResult with fitness=7.713969e-02, inlier_rmse=1.109018e-02, and correspondence_set size of 647716
Access transformation to get result.


## Point-to-point ICP
In general, the ICP algorithm iterates over two steps:

1. Find correspondence set $\mathcal{K}=\{(\mathbf{p}, \mathbf{q})\}$ from target point cloud $\mathbf{P}$, and source point cloud $\mathbf{Q}$ transformed with current transformation matrix $\mathbf{T}$.
2. Update the transformation $\mathbf{T}$ by minimizing an objective function $E(\mathbf{T})$ defined over the correspondence set $\mathcal{K}$.

Different variants of ICP use different objective functions $E(\mathbf{T})$ [\[BeslAndMcKay1992\]](../reference.html#beslandmckay1992) [\[ChenAndMedioni1992\]](../reference.html#chenandmedioni1992) [\[Park2017\]](../reference.html#park2017).

We first show a point-to-point ICP algorithm [\[BeslAndMcKay1992\]](../reference.html#beslandmckay1992) using the objective

\begin{equation}
E(\mathbf{T}) = \sum_{(\mathbf{p},\mathbf{q})\in\mathcal{K}}\|\mathbf{p} - \mathbf{T}\mathbf{q}\|^{2}
\end{equation}

The class `TransformationEstimationPointToPoint` provides functions to compute the residuals and Jacobian matrices of the point-to-point ICP objective. The function `registration_icp` takes it as a parameter and runs point-to-point ICP to obtain the results.

In [12]:
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                      relative_rmse=1e-6,
                                                      max_iteration=100))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
#draw_registration_result(source, target, reg_p2p.transformation)

RegistrationResult with fitness=2.533088e-01, inlier_rmse=9.956842e-03, and correspondence_set size of 2126949
Access transformation to get result.
Transformation is:
[[ 0.996464   -0.04918282 -0.06812155 -0.07512724]
 [ 0.05063016  0.99852347  0.01968439 -0.02689719]
 [ 0.06705283 -0.0230638   0.99748282 -0.0778533 ]
 [ 0.          0.          0.          1.        ]]


In [13]:
source_temp = copy.deepcopy(source)
source_temp.transform(reg_p2p.transformation)
pcd_combined = source_temp + target
#pcd_combined_down = pcd_combined.voxel_down_sample(voxel_size=0.002)
o3d.io.write_point_cloud("PointClouds/Lab_Cam1_Cam2_point-to-point_registration.ply", pcd_combined)

True

## Point-to-plane ICP
The point-to-plane ICP algorithm [\[ChenAndMedioni1992\]](../reference.html#chenandmedioni1992) uses a different objective function

\begin{equation}
E(\mathbf{T}) = \sum_{(\mathbf{p},\mathbf{q})\in\mathcal{K}}\big((\mathbf{p} - \mathbf{T}\mathbf{q})\cdot\mathbf{n}_{\mathbf{p}}\big)^{2},
\end{equation}

where $\mathbf{n}_{\mathbf{p}}$ is the normal of point $\mathbf{p}$. [\[Rusinkiewicz2001\]](../reference.html#rusinkiewicz2001) has shown that the point-to-plane ICP algorithm has a faster convergence speed than the point-to-point ICP algorithm.

`registration_icp` is called with a different parameter `TransformationEstimationPointToPlane`. Internally, this class implements functions to compute the residuals and Jacobian matrices of the point-to-plane ICP objective.

In [14]:
print("Apply point-to-plane ICP")
reg_p2l = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPlane(),
    o3d.pipelinwes.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                      relative_rmse=1e-6,
                                                      max_iteration=100))
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
#draw_registration_result(source, target, reg_p2l.transformation)

Apply point-to-plane ICP
RegistrationResult with fitness=2.165953e-01, inlier_rmse=9.846704e-03, and correspondence_set size of 1818678
Access transformation to get result.
Transformation is:
[[ 9.95918791e-01  4.08132200e-03 -9.01615417e-02 -1.57259714e-01]
 [-8.08137573e-04  9.99340238e-01  3.63102698e-02 -1.23423840e-01]
 [ 9.02502505e-02 -3.60892171e-02  9.95265020e-01 -6.63430062e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [16]:
source_temp = copy.deepcopy(source)
source_temp.transform(reg_p2p.transformation)
pcd_combined = source_temp + target
#pcd_combined_down = pcd_combined.voxel_down_sample(voxel_size=0.002)
o3d.io.write_point_cloud("PointClouds/Lab_Cam1_Cam2_point-to-plane_registration.ply", pcd_combined)

True